# Decision Trees

In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from collections import Counter

## 1. Decision Tree Regression

In [3]:
df = pd.read_csv('study_score.csv')
df

,StudyHours,SleepHours,CoffeeCups,Score,Pass
0,1,9,0,3,0
1,2,8,1,5,0
2,3,7,1,8,1
3,4,7,2,12,1
4,5,6,2,15,1
5,6,6,3,14,1
6,7,5,3,10,1
7,8,5,4,7,0
8,9,4,4,4,0
9,10,4,5,2,0


In [127]:
# TODO: Write a regression tree class that can build tree and make prediction
class RegressionTree:
    def __init__(self, data, candidates: list, target: str, min_samples=5, depth=0, max_depth=3):
        self.tree = self._build_tree(data, candidates, target, min_samples, depth, max_depth)

    def _find_best_feature(self, data, candidates, target):
        best_feature = None
        best_threshold = None
        best_var = np.inf

        for feature in candidates:
            sorted_data = data.sort_values(by=feature)
            values = list(sorted_data[feature])
            targets = list(sorted_data[target])
            thresholds = []

            for i in range(len(values)-1):
                if values[i] != values[i+1]:
                    t = (values[i] + values[i+1]) / 2
                    thresholds.append(t)

            for t in thresholds:
                left_split = [targets[i] for i in range(len(values)) if values[i] <= t]
                right_split = [targets[i] for i in range(len(values)) if values[i] > t]

                left_mean = sum(left_split) / len(left_split) if len(left_split) > 0 else 0
                right_mean = sum(right_split) / len(right_split) if len(right_split) > 0 else 0

                left_var = sum([(val - left_mean)**2 for val in left_split]) / len(left_split) if len(left_split) > 0 else 0
                right_var = sum([(val - right_mean)**2 for val in right_split]) / len(right_split) if len(right_split) > 0 else 0

                weighted_sum = (len(left_split) * left_var) / len(values) + (len(right_split) * right_var) / len(values)

                if weighted_sum < best_var:
                    best_feature = feature
                    best_threshold = t
                    best_var = weighted_sum

        return best_feature, best_threshold, best_var

    def _build_tree(self, data, candidates, target, min_samples, depth, max_depth):
        if len(data[target]) < min_samples or depth >= max_depth:
            return {"type": "leaf", "prediction": sum(data[target]) / len(data[target])}

        best_feature, best_threshold, best_var = self._find_best_feature(data, candidates, target)

        if best_feature is None:
            return {"type": "leaf", "prediction": sum(data[target]) / len(data[target])}

        left_df = data[data[best_feature] <= best_threshold]
        right_df = data[data[best_feature] > best_threshold]

        left_subtree = self._build_tree(left_df, candidates, target, min_samples, depth + 1, max_depth)
        right_subtree = self._build_tree(right_df, candidates, target, min_samples, depth + 1, max_depth)

        return {"type": "node", "feature": best_feature, "threshold": best_threshold, "left": left_subtree, "right": right_subtree}

    def predict(self, tree, sample: dict):
        if tree["type"] == "leaf":
            return tree["prediction"]

        feature = tree["feature"]
        threshold = tree["threshold"]

        if sample[feature] <= threshold:
            return self.predict(tree["left"], sample)
        else:
            return self.predict(tree["right"], sample)

In [128]:
tree = RegressionTree(df, ['CoffeeCups', 'SleepHours', 'StudyHours'], 'Score')
print(tree.predict(tree.tree, {"CoffeeCups": 3, "SleepHours": 6, "StudyHours": 10}))
print(tree.predict(tree.tree, {"CoffeeCups": 2, "SleepHours": 1, "StudyHours": 2}))
print(tree.predict(tree.tree, {"CoffeeCups": 0, "SleepHours": 8, "StudyHours": 6}))

13.666666666666666
3.0
5.333333333333333


In [160]:
sk_reg = DecisionTreeRegressor(max_depth=3, min_samples_split=5)
sk_reg.fit(df[['CoffeeCups', 'SleepHours', 'StudyHours']], df["Score"])
print(sk_reg.predict(pd.DataFrame([[3, 6, 10], [2, 1, 2], [0, 8, 6]], columns=['CoffeeCups', 'SleepHours', 'StudyHours'])))

[13.66666667  3.          5.33333333]


## 2. Decision Classification

In [162]:
# TODO: Write a classificatin tree class that can build tree and make prediction
class ClassificationTree:
    def __init__(self, data, candidates: list, target: str, min_samples=5, depth=0, max_depth=3):
        self.tree = self._build_tree(data, candidates, target, min_samples, depth, max_depth)

    def _find_best_feature(self, data, candidates, target):
        best_gini = np.inf
        best_feature = None
        best_threshold = None

        for feature in candidates:
            sorted_data = data.sort_values(by=feature)
            values = list(sorted_data[feature])
            targets = list(sorted_data[target])
            thresholds = []

            for i in range(len(values)-1):
                if values[i] != values[i+1]:
                    t = (values[i] + values[i+1]) / 2
                    thresholds.append(t)

            for t in thresholds:
                left_split = [targets[i] for i in range(len(values)) if values[i] <= t]
                right_split = [targets[i] for i in range(len(values)) if values[i] > t]

                left_gini = 1.0 - sum((count / len(left_split)) ** 2 for count in Counter(left_split).values()) if len(left_split) != 0 else 0
                right_gini = 1.0 - sum((count / len(right_split)) ** 2 for count in Counter(right_split).values()) if len(right_split) != 0 else 0

                weighted_gini = (len(left_split) * left_gini + len(right_split) * right_gini) / len(values)

                if weighted_gini < best_gini:
                    best_feature = feature
                    best_threshold = t
                    best_gini = weighted_gini

        return best_feature, best_threshold, best_gini

    def _build_tree(self, data, candidates, target, min_samples, depth, max_depth):
        if len(data[target]) < min_samples or depth >= max_depth:
            return {"type": "leaf", "prediction": max(set(data[target]), key=list(data[target]).count)}

        best_feature, best_threshold, best_gini = self._find_best_feature(data, candidates, target)

        if best_feature is None:
            return {"type": "leaf", "prediction": max(set(data[target]), key=list(data[target]).count)}

        left_df = data[data[best_feature] <= best_threshold]
        right_df = data[data[best_feature] > best_threshold]

        left_subtree = self._build_tree(left_df, candidates, target, min_samples, depth + 1, max_depth)
        right_subtree = self._build_tree(right_df, candidates, target, min_samples, depth + 1, max_depth)

        return {"type": "node", "feature": best_feature, "threshold": best_threshold, "left": left_subtree, "right": right_subtree}

    def predict(self, tree, sample: dict):
        if tree["type"] == "leaf":
            return tree["prediction"]

        feature = tree["feature"]
        threshold = tree["threshold"]

        if sample[feature] <= threshold:
            return self.predict(tree["left"], sample)
        else:
            return self.predict(tree["right"], sample)

In [163]:
tree = ClassificationTree(df, ['StudyHours', 'SleepHours', 'CoffeeCups'], 'Pass')
print(tree.predict(tree.tree, {"CoffeeCups": 3, "SleepHours": 6, "StudyHours": 10}))
print(tree.predict(tree.tree, {"CoffeeCups": 2, "SleepHours": 1, "StudyHours": 2}))
print(tree.predict(tree.tree, {"CoffeeCups": 0, "SleepHours": 8, "StudyHours": 6}))

0
0
1


In [166]:
sk_clf = DecisionTreeClassifier(max_depth=3, min_samples_split=5)
sk_clf.fit(df[['CoffeeCups', 'SleepHours', 'StudyHours']], df["Pass"])
print(sk_clf.predict(pd.DataFrame([[3, 6, 10], [2, 1, 2], [0, 8, 6]], columns=['CoffeeCups', 'SleepHours', 'StudyHours'])))

[0 0 1]
